In [1]:
import numpy as np
from scipy import linalg
np.set_printoptions(suppress=True)
A = np.array([[1, 2, 0], [2, 0, 2]]).astype(np.float32)
A

array([[1., 2., 0.],
       [2., 0., 2.]], dtype=float32)

# 2. Singular Value Decomposition dla macierzy 2x3

In [2]:
def svd(A):
    AAT = A @ A.T
    ATA = A.T @ A
    S = calc_S(AAT)
    U = calc_U(AAT)
    V = calc_V(A, U, S)
    return U.astype(np.float32), S.astype(np.float32), V.astype(np.float32)
    
def calc_S(A):
    lambdas = np.sort(linalg.eigvals(A))[::-1]
    return np.sqrt(lambdas)

def calc_U(A):
    lambdas = np.sort(linalg.eigvals(A))[::-1]
    
    A_lambda1 = A - np.eye(A.shape[0]) * lambdas[0]
    A_lambda1 = np.array([-A_lambda1[0, 1], A_lambda1[0, 0]])
    u1 = A_lambda1 / A_lambda1.min()
    u1 = np.expand_dims((1/linalg.norm(u1) * u1).T, 1)
    
    A_lambda1 = A - np.eye(A.shape[0]) * lambdas[1]
    A_lambda1 = np.array([-A_lambda1[0, 1], A_lambda1[0, 0]])
    u2 = A_lambda1 / A_lambda1.min()
    u2 = np.expand_dims((1/linalg.norm(u2) * u2).T, 1)
    
    return np.concatenate([u1, u2], 1)

def calc_V(A, U, S):
    v1 = 1/S[0] * A.T @ U[:,0]
    v2 = 1/S[1] * A.T @ U[:,1]
    v1 = np.expand_dims(v1, 0)
    v2 = np.expand_dims(v2, 0)
    
    ATA = A.T @ A
    lambdas = np.sort(linalg.eigvals(ATA))[::-1]
    A_lambda1 = ATA - np.eye(ATA.shape[0]) * lambdas[2]
    #fix v11=1 and solve rest
    v3_23 = linalg.solve(A_lambda1[:-1, 1:], -A_lambda1[:-1, 0])
    A_lambda1 = np.array([1, v3_23[0], v3_23[1]])
    v3 = A_lambda1 / A_lambda1.min()
    v3 = np.expand_dims((1/linalg.norm(v3) * v3), 0)
    
    return np.concatenate([v1, v2, v3])

In [3]:
svd(A)

/home/andrzej/anaconda3/envs/ed/lib/python3.7/site-packages/ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys


(array([[ 0.4472136,  0.8944272],
        [ 0.8944272, -0.4472136]], dtype=float32),
 array([3., 2.], dtype=float32),
 array([[ 0.745356  ,  0.2981424 ,  0.5962848 ],
        [ 0.        ,  0.8944272 , -0.4472136 ],
        [-0.6666667 ,  0.33333334,  0.6666667 ]], dtype=float32))

# Check if result is correct

In [4]:
linalg.svd(A)

(array([[ 0.44721377,  0.8944272 ],
        [ 0.8944272 , -0.44721368]], dtype=float32),
 array([3.       , 1.9999998], dtype=float32),
 array([[ 0.7453559 ,  0.29814234,  0.59628487],
        [ 0.        ,  0.8944272 , -0.44721362],
        [-0.6666667 ,  0.33333337,  0.6666666 ]], dtype=float32))